# Comparison TARTAC00 vs. TARTDL00, diastereomeric relationship
Based on Respons_2_Norwid, changes start with line 4.

In [1]:
import numpy as np
import os, sys
if os.path.isfile(os.getenv("HOME") + '/software/ArbAlign/hungarian.cpython-36m-x86_64-linux-gnu.so'):
    sys.path.append(os.getenv("HOME") + '/software/ArbAlign/')
    import hungarian
    linear_sum_assignment =  hungarian.lap
else:
    from scipy.optimize import linear_sum_assignment

from scipy.spatial.distance import cdist

AXIS_SWAPS       = np.array([[0,1,2],[0,2,1],[1,0,2],[1,2,0],[2,0,1],[2,1,0]])
mask_swaps       =           [  1   ,  -1   ,  -1   ,   1   ,   1   ,  -1  ]
AXIS_REFLECTIONS = np.array([[1,1,1],[-1,1,1],[1,-1,1],[1,1,-1],[-1,-1,1],[-1,1,-1],[1,-1,-1],[-1,-1,-1]])
mask_reflections =           [  1   ,   -1   ,   -1   ,  -1    ,     1   ,    1    ,    1    ,    -1   ]
_rmsd = lambda V, W: np.sqrt(sum([(v[i]-w[i])**2.0 for i in range(len(V[0])) for v, w in zip(V, W)])/len(V))
centroid = lambda X: X.mean(axis=0)

# Extract element symbols and atomic coordinates from PDB string or filename
def parse_pdb(pdb, noHydrogens=True):
    """
    Reads an pdf file into lists
    filename - name of xyz file
    noHydrogens - if true, hydrogens are ignored; if not, hydrogens are included

    Returns a tuple (a, b)
    where a is a list of coordinate labels and b is a set of coordinates
    (i.e) a = ["O", "H", "H"], b = [[x0,y0,z0],[x1,y1,z1],[x2,y2,z2]]
    """
    pdb = pdb.split('\n')
    if len(pdb) == 1:
        with open(pdb[0], 'r') as fr:
            pdb = fr.readlines()
    labels = []
    coords = []
    for i in [i for i in pdb if i.find('HETATM') == 0 or i.find('ATOM') == 0 ]:
        # ATOM      1  OXT GLY A   3       2.261  -2.612   1.127  1.00  0.00           O
        # HETATM    1  C1  UNL     1      -7.227   0.602  -0.445  1.00  0.00           C
        ElementSymbol = i[76:78].strip().upper()
        #print(noHydrogens)
        if noHydrogens and ElementSymbol == "H":
            continue
        else:
            labels.append(ElementSymbol)
            coords.append([i[30:38],i[38:46],i[46:54]])
    #print(unsorted_labels, np.array(unsorted_coords, dtype=float), NA )
    return np.asarray(labels), np.array(coords, dtype=float)

# Rotate the coordinates of probe molecule according to those of reference molecule
def kabsch_rmsd(prb, ref):
    # Computation of the covariance matrix
    C = np.dot(np.transpose(prb), ref)

    # Computation of the optimal rotation matrix
    V, S, W = np.linalg.svd(C)

    # Ensure a right-handed coordinate system
    if (np.linalg.det(V) * np.linalg.det(W)) < 0.0:
        S[-1] = -S[-1]
        V[:, -1] = -V[:, -1]

    # Create Rotation matrix U
    U = np.dot(V, W)
    prb = np.dot(prb, U)
    return _rmsd(prb, ref), prb

def reorder_hungarian(p_atoms, q_atoms, p_coord, q_coord):
    """
    Re-orders the input atom list and xyz coordinates using the Hungarian
    method (using optimized column results)

    Parameters
    ----------
    p_atoms : array
        (N,1) matrix, where N is points holding the atoms' names
    p_atoms : array
        (N,1) matrix, where N is points holding the atoms' names
    p_coord : array
        (N,D) matrix, where N is points and D is dimension
    q_coord : array
        (N,D) matrix, where N is points and D is dimension

    Returns
    -------
    view_reorder : array
             (N,1) matrix, reordered indexes of atom alignment based on the
             coordinates of the atoms

    """

    # Find unique atoms
    unique_atoms = np.unique(p_atoms)

    # generate full view from q shape to fill in atom view on the fly
    view_reorder = np.zeros(q_atoms.shape, dtype=int)
    view_reorder -= 1
    for atom in unique_atoms:
        p_atom_idx, = np.where(p_atoms == atom)
        q_atom_idx, = np.where(q_atoms == atom)

        A_coord = p_coord[p_atom_idx]
        B_coord = q_coord[q_atom_idx]

        # should be kabasch here i think
        distances = cdist(A_coord, B_coord, 'euclidean')

        # Perform Hungarian analysis on distance matrix between atoms of 1st
        # structure and trial structure
        _, view = linear_sum_assignment(distances)

        view_reorder[p_atom_idx] = q_atom_idx[view]

    return view_reorder

# Description of the rotated molecule in XYZ format
def get_xyz(atoms, V, title="", decimals=8):
    """
    Print coordinates V with corresponding atoms to stdout in XYZ format.
    Parameters
    ----------
    atoms : list
        List of atomic types
    V : array
        (N,3) matrix of atomic coordinates
    title : string (optional)
        Title of molecule
    decimals : int (optional)
        number of decimals for the coordinates

    Return
    ------
    output : str
        Molecule in XYZ format

    """
    N, D = V.shape

    fmt = "{:2s}" + (" {:15."+str(decimals)+"f}")*3

    out = list()
    out += [str(N)]
    out += [title]

    for i in range(N):
        atom = atoms[i]
        atom = atom[0].upper() + atom[1:]
        out += [fmt.format(atom, V[i, 0], V[i, 1], V[i, 2])]

    return "\n".join(out)

def _align(ref_symbol, prb_symbol, ref_coord, prb_coord, ref_cent,
          reorder=False, keep_stereo=True):

    min_rmsd   = np.inf
    min_review = None
    tmp_review = None

    for swap, i in zip(AXIS_SWAPS, mask_swaps):
        for reflection, j in zip(AXIS_REFLECTIONS, mask_reflections):
            if keep_stereo and  i * j == -1: continue # skip enantiomers

            # Swap, reflect, and translate the probe molecule
            tmp_coord  = prb_coord[:, swap]
            tmp_coord  = np.dot(tmp_coord, np.diag(reflection))
            tmp_coord -= centroid(tmp_coord)

            # Reorder the modifed probe
            if reorder:
                tmp_review = reorder_hungarian(ref_symbol, prb_symbol, ref_coord, tmp_coord)
                tmp_coord = tmp_coord[tmp_review]

            # Rotation
            this_rmsd, tmp_coord  = kabsch_rmsd(tmp_coord, ref_coord)

            if this_rmsd < min_rmsd:
                min_rmsd    = this_rmsd
                min_review  = tmp_review
                min_coord   = tmp_coord
                i_j_product = i * j 

    if reorder:
        prb_symbol = prb_symbol[min_review]
    prb_coord = min_coord + ref_cent

    prb_xyz = get_xyz(prb_symbol, prb_coord, title="aligned", decimals=3)
    return min_rmsd, prb_xyz, i_j_product

def align(ref_pdb, prb_pdb, noHydrogens=True, reorder=False, keep_stereo=True):
    ref_symbol, ref_coord = parse_pdb(ref_pdb, noHydrogens)
    prb_symbol, prb_coord = parse_pdb(prb_pdb, noHydrogens)
    if len(ref_symbol) != len(prb_symbol): raise
    ref_cent = centroid(ref_coord)
    prb_cent = centroid(prb_coord)
    ref_coord -= ref_cent
    prb_coord -= prb_cent
    result = _align(ref_symbol, prb_symbol,
                    ref_coord, prb_coord, ref_cent,
                    reorder, keep_stereo)
    return result

if __name__ == "__main__":
    ref = '''REMARK   1 File created by GaussView 5.0.9
HETATM    1  C           0      -0.084  -0.121  -2.952                       C
HETATM    2  H           0       0.329  -1.099  -3.088                       H
HETATM    3  H           0       0.335   0.547  -3.675                       H
HETATM    4  H           0      -1.146  -0.162  -3.076                       H
HETATM    5  C           0       0.248   0.383  -1.535                       C
HETATM    6  H           0      -0.163   1.362  -1.400                       H
HETATM    7  H           0       1.310   0.422  -1.410                       H
HETATM    8  C           0      -0.357  -0.576  -0.494                       C
HETATM    9  H           0       0.053  -1.555  -0.629                       H
HETATM   10  H           0      -1.419  -0.615  -0.619                       H
HETATM   11  C           0      -0.025  -0.072   0.923                       C
HETATM   12  N           0      -0.589   1.273   1.109                       N
HETATM   13  H           0      -1.548   1.197   1.383                       H
HETATM   14  H           0      -0.077   1.754   1.821                       H
HETATM   15  O           0       1.394  -0.021   1.090                       O
HETATM   16  H           0       1.608  -0.064   2.025                       H
HETATM   17  F           0      -0.556  -0.913   1.836                       F
END '''
    prb = '''TITLE       Required
REMARK   1 File created by GaussView 5.0.9
HETATM    1  C           0     -16.365  -0.185   0.225                       C
HETATM    2  H           0     -17.102  -0.670  -0.381                       H
HETATM   11  C           0     -14.053  -3.247   0.772                       C
HETATM   12  N           0     -13.039  -2.580   1.602                       N
HETATM   13  H           0     -13.172  -2.836   2.560                       H
HETATM    9  H           0     -16.088  -2.904   0.178                       H
HETATM   10  H           0     -15.712  -2.333   1.785                       H
HETATM   16  H           0     -13.006  -4.136  -0.641                       H
#HETATM   17  F           0     -14.311  -4.478   1.268                       F
HETATM   17  F           0     -14.301  -4.478   1.268                       F
HETATM   14  H           0     -12.127  -2.862   1.306                       H
HETATM   15  O           0     -13.570  -3.362  -0.569                       O
HETATM    3  H           0     -16.167   0.791  -0.166                       H
HETATM    4  H           0     -16.728  -0.102   1.229                       H
HETATM    5  C           0     -15.067  -1.015   0.216                       C
HETATM    6  H           0     -14.330  -0.528   0.821                       H
HETATM    7  H           0     -14.705  -1.100  -0.787                       H
HETATM    8  C           0     -15.351  -2.418   0.781                       C
END '''
    min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=True, keep_stereo=False)
    print('mininum rmsd:',min_rmsd,'\n')
    print(prb_xyz)

mininum rmsd: 0.0003733837687950025 

7
aligned
C           -0.084          -0.121          -2.952
C            0.248           0.383          -1.535
C           -0.357          -0.576          -0.494
C           -0.025          -0.072           0.923
N           -0.589           1.273           1.109
O            1.394          -0.021           1.090
F           -0.556          -0.913           1.836


# TARTAC00.pdb as ref (reference molecule)
# TARTDL00.pdb as prb (probe molecule)

In [2]:
ref = '''CRYST1    7.710    6.000    6.230  90.00 100.10  90.00 P 1 21 1      2
TITLE OLEX2 export TARTAC00O-noH.pdb
ATOM      1   C1                 3.051   0.095  -4.396  1.00  3.95           C  
ATOM      2   C2                 4.461  -0.041  -4.961  1.00  3.95           C  
ATOM      3   C3                 2.439   1.364  -5.023  1.00  3.95           C  
ATOM      4   C4                 0.989   1.517  -4.603  1.00  3.95           C  
ATOM      5   O5                 3.143   0.206  -3.004  1.00  3.95           O  
ATOM      6   O6                 4.429  -0.195  -6.268  1.00  3.95           O  
ATOM      7   O7                 5.455   0.030  -4.290  1.00  3.95           O  
ATOM      8   O8                 3.199   2.504  -4.701  1.00  3.95           O  
ATOM      9   O9                 0.307   0.475  -5.036  1.00  3.95           O  
ATOM     10  O10                 0.543   2.443  -3.989  1.00  3.95           O    
'''
prb = '''CRYST1    8.060    9.600    4.850  70.38  97.20 112.47 P -1          2
TITLE OLEX2 export, TARTDL00O-noH.pdb
ATOM      1   O1                -3.274   2.553   1.873  1.00  3.95           O  
ATOM      2   O2                -3.623   4.704   1.964  1.00  3.95           O  
ATOM      3   O3                -5.679   4.429   3.655  1.00  3.95           O  
ATOM      4   O4                -6.606   3.162   1.325  1.00  3.95           O  
ATOM      5   O5                -8.383   2.115   3.015  1.00  3.95           O  
ATOM      6   O6                -6.840   1.593   4.523  1.00  3.95           O  
ATOM      7   C7                -3.973   3.564   2.193  1.00  3.95           C  
ATOM      8   C8                -5.064   3.230   3.061  1.00  3.95           C  
ATOM      9   C9                -6.116   2.394   2.330  1.00  3.95           C  
ATOM     10  C10                -7.227   2.129   3.381  1.00  3.95           C   
'''

# Test options reorder and keep_stereo

In [8]:
min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=False, keep_stereo=False)
print('mininum rmsd:',min_rmsd,'\n') 

mininum rmsd: 2.2469098354289057 



In [4]:
min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=False, keep_stereo=True)
print('mininum rmsd:',min_rmsd,'\n') 

mininum rmsd: 2.379347683339732 



In [5]:
min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=True, keep_stereo=False)
print('rmsd', min_rmsd, '\ni_j_product: ',i_j_product, '\n\n', prb_xyz)

rmsd 0.16249943738098105 
i_j_product:  1 

 10
aligned
C            3.050           0.080          -4.369
C            4.483           0.055          -4.964
C            2.363           1.278          -5.027
C            1.019           1.431          -4.552
O            3.108           0.337          -3.038
O            4.572           0.046          -6.281
O            5.405          -0.271          -4.248
O            3.158           2.502          -4.828
O            0.254           0.494          -4.937
O            0.605           2.445          -4.027


In [6]:
min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=True, keep_stereo=True)
print('rmsd', min_rmsd, '\ni_j_product: ',i_j_product, '\n\n', prb_xyz)

rmsd 0.16249943738098105 
i_j_product:  1 

 10
aligned
C            3.050           0.080          -4.369
C            4.483           0.055          -4.964
C            2.363           1.278          -5.027
C            1.019           1.431          -4.552
O            3.108           0.337          -3.038
O            4.572           0.046          -6.281
O            5.405          -0.271          -4.248
O            3.158           2.502          -4.828
O            0.254           0.494          -4.937
O            0.605           2.445          -4.027


# We see the option keep_stereo is responsible for the rmsd difference. 

The models are derived from different cif lines of the CSD data base.  The TARTAC datum refers to D-(-)-(2S,3S)-tartaric acid.  TARTDL refers to (2R, 3S)-tartaric acid, wich is both diastereomer and meso-compound with the former.  It was expected that there would no difference in using either --use-reflections or --use-reflections-keep-stereo.

However, I notice calculate_rmsd.py from the CLI yields a RMSD of 1.419; then -- with either of the reflection modi -- a RMSD of 0.1625.  Again, none of the three sets of coordinates of the moved motif provided by calculate_rmsd.py on the CLI match the ones provied here.  Quite contrary to expectation, the superposition of the original molecule with the then moved second one /suggests/ a much nicer fit in calculate_rmsd.py default mode, than in each of the others.